# I. $LU$ factorization of a square matrix

When we premultply $A$ by lower triangular elemetary matrices $\Lambda_j$ it is trasfrommed in an  upper triangular matrix $U$

$$  \Lambda_{n-1}  \ldots\Lambda_2  \Lambda_1 A  = U$$

$$  A  = (\Lambda_{n-1}  \ldots\Lambda_2  \Lambda_2 )^{-1} U$$

The inverse of a product of matrices is the revesre product of inverses
$$  A  = (\Lambda_1^{-1}  \Lambda_2^{-1} \ldots  \Lambda_{n-1}^{-1}) U$$

and inverse of a Lower Triangular Elemetary Matrix is minus the matrix $\Lambda_j^{-1} = - \Lambda_j$, so 

$$ A  =  (-\Lambda_1) (-\Lambda_2) \ldots   (-\Lambda_{n-1})  U$$

So the $LU$ column pivot factorization is 
$$  A  = L U$$
with
$$ U = \Lambda_{n-1}  \ldots\Lambda_2  \Lambda_1 A  $$
an upper triangular matrix
$$ L  =  \Lambda_1^{-1}  \Lambda_2^{-1} \ldots  \Lambda_{n-1}^{-1} $$
an lower triangular matrix.

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [ ]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [ ]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

np.linalg.matrix_rank(a)

6

In [ ]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)

In [ ]:
L, u = diy_lu(a)

print(L, "\n")
print(u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(L@u - a)

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  2.220e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -4.528e-16  0.000e+00  6.939e-18  8.080e-04  1.902e-03]
 [ 0.000e+00  4.123e-16  0.000e+00 -1.634e-17  0.000e+00 -1.585e-05]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00 -1.110e-16  1.110e-16  1.110e-16 -5.551e-17]
 [ 0.000e+00  0.000e+00  3.331e-16 -2.220e-16 -5.551e-17  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00 -1.110e-16 -1.665e-16  0.000e+00]
 

In [ ]:
a

array([[3.   , 3.   , 3.   , 3.   , 3.   , 3.   ],
       [3.   , 1.875, 1.364, 1.071, 0.882, 0.75 ],
       [3.   , 1.364, 0.882, 0.652, 0.517, 0.429],
       [3.   , 1.071, 0.652, 0.469, 0.366, 0.3  ],
       [3.   , 0.882, 0.517, 0.366, 0.283, 0.231],
       [3.   , 0.75 , 0.429, 0.3  , 0.231, 0.188]])

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [ ]:
a1 = a.copy()
a1[1, 1] = 3


In [ ]:
a1

array([[3.   , 3.   , 3.   , 3.   , 3.   , 3.   ],
       [3.   , 3.   , 1.364, 1.071, 0.882, 0.75 ],
       [3.   , 1.364, 0.882, 0.652, 0.517, 0.429],
       [3.   , 1.071, 0.652, 0.469, 0.366, 0.3  ],
       [3.   , 0.882, 0.517, 0.366, 0.283, 0.231],
       [3.   , 0.75 , 0.429, 0.3  , 0.231, 0.188]])

Resulting matix still has full rank, but the naive LU routine breaks down.

In [ ]:
np.linalg.matrix_rank(a1)

6

In [ ]:
l, u = diy_lu(a1)

print(l, u)

[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in matmul
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in matmul
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [ ]:
# ... ENTER YOUR CODE HERE ...

def revisionCero(x):
  notCero=True
  for item in range(x.shape[0]):
    if np.linalg.det(x[:item+1,:item+1]) == 0:
      notCero= False
      break
  return notCero

print("Para a:",revisionCero(a),"\n y para b:",revisionCero(a1))






Para a: True 
 y para b: False


### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

# 2. $LU$ factorization column pivoting
When we premultply $A$ by elementary permutation matricex$P_j$( to find a good pivot) and the premultply by lower triangular elemetary matrices $\Lambda_j$  it is trasfrommed in an  upper triangular matrix $U$

$$  \Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1 A  = U$$
$$  A  = (\Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1)^{-1} U$$

with

\begin{array}{ll}L  &=  (\Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1)^{-1}\\
&= P_1^{-1} \Lambda_1^{-1} P_2^{-1} \Lambda_2^{-1} \ldots  P_{n-1}^{-1}\\
 &= P_1^{t} (-\Lambda_1) P_2^{t} (-\Lambda_2) \ldots  P_{n-1}^{t} (-\Lambda_{n-1})
\end{array}



Because the inverse a of a Lower Triangular Elemetary Matrix is minus the matrix $\Lambda_i^{-1} = - \Lambda_i$ and the inverse of a Permutation Matrix (in particular an elementary permutation matrix)  is its transpose [math.stackexchange](
        https://math.stackexchange.com/questions/98549/the-transpose-of-a-permutation-matrix-is-its-inverse#:~:text=Taking%20the%20transpose%20of%20P,Pt%3DP%E2%88%921.)  $P_i^{-1}=P_i^{t}$, so


If we use 
$$  \hat{L}  = (\Lambda_{n-1}  \ldots \Lambda_2\Lambda_1)^{-1}$$
The inverse of a product of matrices is the revesre product of inverses 
$$ \hat{L} = \Lambda_1^{-1}\Lambda_2^{-1} \ldots  \Lambda_{n-1}^{-1} $$
the inverse a of a Lower Triangular Elemetary Matrix is minus the matrix $\Lambda_i^{-1} = - \Lambda_i$
$$ \hat{L} = (-\Lambda_1)  (-\Lambda_2) \ldots  (-\Lambda_{n-1}) $$

Then
$$    A_\pi  = \hat{L} U$$
with $A_\pi$ equal to $A$ with rows permutated acoording to some permutation $\pi$

Equiavlently 
$$   A  = (\hat{L}U)_{\pi'}$$
 $(\hat{L}U)_{\pi'}$ equal to $\hat{L}U$ with rows permutated acoording to some permutation $\pi'$

In [ ]:
# ... ENTER YOUR CODE HERE ...

def diy_lu_Mod(a): 

    N = a.shape[0]
    u = a.copy()
    L = np.eye(N)
    per=np.eye(N)

    for j in range(N-1):
        lam=np.eye(N)
        permutar = np.eye(N)
        pivote=np.argmax(abs(u[j:,j]))+j
        permutar[j],permutar[pivote]=permutar[pivote],permutar[j].copy()
        u=permutar @ u
        gamma = u[j+1:,j]/u[j,j]
        lam[j+1:,j] = -gamma
        u = lam @ u
        lam[j+1:,j] = gamma
        per= per @ permutar
        L = L@permutar.T@lam
    return per,per.T@L,u


def productoMatrices(a):
  x=a[0].shape[0]
  y=np.eye(x)
  for item in a:
    y=y @ item
  return y

matriz1= diy_lu_Mod(a1)
matriz2= diy_lu_Mod(a)


print("Para la matriz a1: \n",productoMatrices(matriz1)-a1,"\n")
print("Para la matriz a: \n",productoMatrices(matriz2)-a)

Para la matriz a1: 
 [[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  2.220e-16  1.110e-16 -2.776e-16]
 [ 0.000e+00  0.000e+00  1.110e-16 -1.665e-16 -2.776e-16 -5.551e-17]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  3.331e-16 -2.776e-17]
 [ 0.000e+00  0.000e+00 -1.665e-16 -1.665e-16  5.551e-17  0.000e+00]] 

Para la matriz a: 
 [[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  2.220e-16  1.110e-16 -2.776e-16]
 [ 0.000e+00  0.000e+00  1.110e-16 -1.665e-16 -2.776e-16 -5.551e-17]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  3.331e-16  0.000e+00]
 [ 0.000e+00  0.000e+00 -1.665e-16 -1.665e-16  5.551e-17  0.000e+00]]


In [ ]:
N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

L, u, P = diy_lu_Mod(a)

print("L\n",L, "\n")
print("u\n",u, "\n")
print("L@u\n", L@u, "\n")
print("a\n",a, "\n")
print("P\n", P, "\n")
print("P@L@u\n", P@L@u, "\n")
print("a\n",a, "\n")

L
 [[1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0.]] 

u
 [[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    0.5   1.    0.    0.    0.   ]
 [1.    0.727 0.706 1.    0.    0.   ]
 [1.    0.857 0.41  0.835 1.    0.   ]
 [1.    0.941 0.178 0.426 0.789 1.   ]] 

L@u
 [[1.    0.    0.    0.    0.    0.   ]
 [1.    0.5   1.    0.    0.    0.   ]
 [1.    0.727 0.706 1.    0.    0.   ]
 [1.    0.857 0.41  0.835 1.    0.   ]
 [1.    0.941 0.178 0.426 0.789 1.   ]
 [1.    1.    0.    0.    0.    0.   ]] 

a
 [[3.    3.    3.    3.    3.    3.   ]
 [3.    1.875 1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 0.188]] 

P
 [[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -2.250e+00 -2.571e+00 -2.700e+00 -2.769e+00 -2.812e+00]
 [ 0.

In [ ]:
a1 = a.copy()
a1[1, 1] = 3

L, u , P = diy_lu_Mod(a1)

print("L\n",L, "\n")
print("u\n",u, "\n")
print("L@u\n", L@u, "\n")
print("a\n",a, "\n")
print("P\n", P, "\n")
print("P@L@u\n", P@L@u, "\n")
print("a\n",a, "\n")

L
 [[1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0.]] 

u
 [[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    0.    1.    0.    0.    0.   ]
 [1.    0.727 0.151 1.    0.    0.   ]
 [1.    0.857 0.088 0.514 1.    0.   ]
 [1.    0.941 0.038 0.208 0.641 1.   ]] 

L@u
 [[1.    0.    0.    0.    0.    0.   ]
 [1.    0.    1.    0.    0.    0.   ]
 [1.    0.727 0.151 1.    0.    0.   ]
 [1.    0.857 0.088 0.514 1.    0.   ]
 [1.    0.941 0.038 0.208 0.641 1.   ]
 [1.    1.    0.    0.    0.    0.   ]] 

a
 [[3.    3.    3.    3.    3.    3.   ]
 [3.    1.875 1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 0.188]] 

P
 [[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -2.250e+00 -2.571e+00 -2.700e+00 -2.769e+00 -2.812e+00]
 [ 0.

In [ ]:
a2 = np.array([[4,3,1,2], [5,7,0,1], [9,9,3,8], [8,2,4,7]])

L, u, P = diy_lu_Mod(a2)

print("L\n",L, "\n")
print("u\n",u, "\n")
print("L@u\n", L@u, "\n")
print("a\n",a, "\n")
print("P\n", P, "\n")
print("P@L@u\n", P@L@u, "\n")
print("a\n",a, "\n")

L
 [[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]] 

u
 [[ 1.     0.     0.     0.   ]
 [ 0.889  1.     0.     0.   ]
 [ 0.556 -0.333  1.     0.   ]
 [ 0.444  0.167  0.455  1.   ]] 

L@u
 [[ 0.444  0.167  0.455  1.   ]
 [ 0.556 -0.333  1.     0.   ]
 [ 1.     0.     0.     0.   ]
 [ 0.889  1.     0.     0.   ]] 

a
 [[3.    3.    3.    3.    3.    3.   ]
 [3.    1.875 1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 0.188]] 

P
 [[ 9.     9.     3.     8.   ]
 [ 0.    -6.     1.333 -0.111]
 [ 0.     0.    -1.222 -3.481]
 [ 0.     0.     0.     0.045]] 

P@L@u
 [[19.111  6.5   13.091  9.   ]
 [-2.099  1.889 -6.     0.   ]
 [-4.317 -3.481  0.     0.   ]
 [ 0.04   0.045  0.     0.   ]] 

a
 [[3.    3.    3.    3.    3.    3.   ]
 [3.    1.875 1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0

# 3. $LU$ factorization column pivoting and reconstruction
When we premultply $A$ by elementary permutation matricex$P_j$( to find a good pivot) and the premultply by lower triangular elemetary matrices $\Lambda_j$  it is trasfrommed in an  upper triangular matrix $U$

$$  \Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1 A  = U$$
$$  A  = (\Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1)^{-1} U$$
$$  A  = (\Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1)^{-1} U$$

The inverse of a product of matrices is the revesre product of inverses 
$$  A  = ( P_1^{-1} \Lambda_1^{-1} P_2^{-1} \Lambda_2^{-1} \ldots  P_{n-1}^{-1}\Lambda_{n-1}^{-1} ) U$$

So the $LU$ column pivot factorization is 
$$  A  = L U$$
with
$$ U = \Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1 A  $$



$$ L  = P_1^{-1} \Lambda_1^{-1} P_2^{-1} \Lambda_2^{-1} \ldots  P_{n-1}^{-1}\Lambda_{n-1}^{-1} $$

the inverse a of a Lower Triangular Elemetary Matrix is minus the matrix $\Lambda_i^{-1} = - \Lambda_i$ and the inverse of a Permutation Matrix (in particular an elementary permutation matrix)  is its transpose [math.stackexchange](
        https://math.stackexchange.com/questions/98549/the-transpose-of-a-permutation-matrix-is-its-inverse#:~:text=Taking%20the%20transpose%20of%20P,Pt%3DP%E2%88%921.)  $P_i^{-1}=P_i^{t}$, so

$$ L  =  P_1^{t} (-\Lambda_1) P_2^{t} (-\Lambda_2) \ldots  P_{n-1}^{t} (-\Lambda_{n-1}) $$


In [ ]:
def diy_lu_column_pivot_reconstruct(a):
  

  """Construct the LU decomposition of the input matrix.
  
  LU decomposition with pivot: work column by column, accumulate elementary triangular matrices L @ np.transpose(Pj) .
  """
  # ... ENTER YOUR CODE HERE ...

  N = a.shape[0]
  u = a.copy()
  L = np.eye(N)

  for j in range(1, N-1):
    lam = np.eye(N)
    gamma = u[j-1:, j] / u[j, j]
    lam[j-1:, j] = -gamma
    u = lam @ u
     
    lam[j-1:, j] = gamma
    L = L @ lam
  return  L, u

In [ ]:
N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

L, u = diy_lu_column_pivot_reconstruct(a)

print("L\n",L, "\n")
print("u\n",u, "\n")
print("L@u\n", L@u, "\n")
print("a\n",a, "\n")

L
 [[1.000e+00 1.600e+00 1.995e+01 6.228e+02 4.605e+04 0.000e+00]
 [0.000e+00 1.000e+00 1.247e+01 3.893e+02 2.878e+04 0.000e+00]
 [0.000e+00 7.273e-01 1.007e+01 3.143e+02 2.324e+04 0.000e+00]
 [0.000e+00 5.714e-01 8.285e+00 2.597e+02 1.920e+04 0.000e+00]
 [0.000e+00 4.706e-01 7.005e+00 2.202e+02 1.628e+04 0.000e+00]
 [0.000e+00 4.000e-01 6.055e+00 1.908e+02 1.411e+04 1.000e+00]] 

u
 [[-1.800e+00 -1.665e-16  8.182e-01  1.286e+00  1.588e+00  1.800e+00]
 [-1.320e+01 -1.875e+00 -4.242e-17  5.124e-01  6.694e-01  7.071e-01]
 [-1.129e+01  0.000e+00  1.094e-01 -7.476e-19 -6.982e-02 -1.075e-01]
 [-1.100e+01  0.000e+00  0.000e+00 -4.069e-03  9.253e-20  4.533e-03]
 [-1.443e-01  0.000e+00  0.000e+00  0.000e+00  8.416e-05  1.585e-04]
 [ 6.152e-02  0.000e+00  0.000e+00  0.000e+00  0.000e+00  1.056e-06]] 

L@u
 [[-1.374e+04 -3.000e+00  3.000e+00 -4.286e-01  5.142e+00  1.091e+01]
 [-8.588e+03 -1.875e+00  1.364e+00 -1.071e+00  2.221e+00  5.694e+00]
 [-6.934e+03 -1.364e+00  1.101e+00 -9.063e-01  1.740e

In [ ]:
a1 = a.copy()
a1[1, 1] = 3

L, u, = diy_lu_column_pivot_reconstruct(a1)

print("L\n",L, "\n")
print("u\n",u, "\n")
print("L@u\n", L@u, "\n")
print("a2\n",a1, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print("L@u - a1\n",L@u - a1, "\n")

L
 [[ 1.000e+00  1.000e+00 -5.194e+00 -4.816e+01 -6.549e+03  0.000e+00]
 [ 0.000e+00  1.000e+00 -5.194e+00 -4.816e+01 -6.549e+03  0.000e+00]
 [ 0.000e+00  4.545e-01 -1.361e+00 -1.262e+01 -1.716e+03  0.000e+00]
 [ 0.000e+00  3.571e-01 -1.226e+00 -1.037e+01 -1.410e+03  0.000e+00]
 [ 0.000e+00  2.941e-01 -1.085e+00 -8.807e+00 -1.197e+03  0.000e+00]
 [ 0.000e+00  2.500e-01 -9.647e-01 -7.653e+00 -1.039e+03  1.000e+00]] 

u
 [[ 0.000e+00  0.000e+00  1.636e+00  1.929e+00  2.118e+00  2.250e+00]
 [ 5.500e+00 -3.000e+00 -1.061e-16 -2.135e-01 -2.789e-01 -2.946e-01]
 [-9.973e+00  0.000e+00 -2.625e-01 -1.069e-17  9.116e-02  1.257e-01]
 [-3.691e+01  0.000e+00  0.000e+00  1.781e-02 -2.112e-18 -1.400e-02]
 [-2.649e-01  0.000e+00  0.000e+00  0.000e+00 -1.645e-04 -2.722e-04]
 [ 1.038e-01  0.000e+00  0.000e+00  0.000e+00  0.000e+00 -1.846e-06]] 

L@u
 [[ 3.570e+03 -3.000e+00  3.000e+00  8.571e-01  2.442e+00  3.760e+00]
 [ 3.570e+03 -3.000e+00  1.364e+00 -1.071e+00  3.245e-01  1.510e+00]
 [ 9.365e+02 -1.3

In [ ]:
a2 = np.array([[4,3,1], [5,7,0], [9,9,3], [8,2,4]])

L, u, = diy_lu_column_pivot_reconstruct(a2)

print("L\n",L, "\n")
print("u\n",u, "\n")
print("L@u\n", L@u, "\n")
print("a2\n",a2, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print("L@u - a2\n",L@u - a2, "\n")

L
 [[1.    0.429 0.    0.   ]
 [0.    1.    0.    0.   ]
 [0.    1.286 1.    0.   ]
 [0.    0.286 1.333 1.   ]] 

u
 [[ 1.857e+00  1.665e-16  1.000e+00]
 [-5.000e+00 -7.000e+00  0.000e+00]
 [-2.571e+00  0.000e+00 -3.000e+00]
 [ 3.143e+00  0.000e+00  0.000e+00]] 

L@u
 [[-0.286 -3.     1.   ]
 [-5.    -7.     0.   ]
 [-9.    -9.    -3.   ]
 [-1.714 -2.    -4.   ]] 

a2
 [[4 3 1]
 [5 7 0]
 [9 9 3]
 [8 2 4]] 

L@u - a2
 [[ -4.286  -6.      0.   ]
 [-10.    -14.      0.   ]
 [-18.    -18.     -6.   ]
 [ -9.714  -4.     -8.   ]] 

